# Deep Learning Image Transfer (Convolutional Neural Network)

### Importing the libraries

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [2]:
tf.__version__

'2.11.0'

## Data Preprocessing

### Preprocessing the Training set

### Preprocessing the Test set

## Building the CNN

### Initialising the CNN

### Convolution

### Pooling

### Adding a second convolutional layer

### Flattening

### Full Connection

### Output Layer

## Training the CNN

### Compiling the CNN

### Training the CNN on the Training set and evaluating it on the Test set

## Making a single prediction